# Prepare a dataset for use in a training task

**Goals**: show the steps necessary to download, preprocess, and register a dataset for use with AzureML. 

Working with AzureML registered datasets allows you to avoid bundling the dataset with the context you send to each training node (slow) or setting up a persistent mount on your Deepspeed Docker images (more complex and brittle). Additionally, using registered datasets offers the opportunity to log what version of a dataset a model was trained with. This means you can monitor the effects of retraining over time as your data increases or changes. 

A note on terminology: I'll endeavor use the term dataset to refer to the AzureML dataset object stored on AzureML and data when referencing the same information stored locally.

Here we'll be working with the [CoLA](https://nyu-mll.github.io/CoLA/) dataset, on which we'll later be fine-tuning a model. We'll perform the following steps:

- Download the data appropriate for the training task
- Preprocess (tokenize) it
- Save it locally
- Upload the preprocessed data to the AzureML datastore where our AzureML datasets are located
- Register the resulting files as a new AzureML dataset

First, imports

In [ ]:
import yaml
import azureml.core
import datasets
import transformers

## Using an external config.yml

Now we'll load a `src/config.yml` file that tells us the task name and data locations, this is also used by the training functions and so keeps everything in one place. More information about it is available in the companion `Train model` notebook.

In [ ]:
with open('src/config.yml', 'r') as f:
    config = yaml.safe_load(f)

## Load or mount data 

The data itself will be downloaded by the `datasets` libraries from [Huggingface](https://huggingface.co/). Loading this public dataset is quite simple but this is where you'd locally mount or otherwise access a proprietary dataset. 

In [ ]:
data = datasets.load_dataset("glue", config['task'])

## Preprocess data

We will preprocess our data before we save it. In this case it means tokenizing the data to prepare it for the model specified in the `src/config.yml` file.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(config['model'])
def tokenizer_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)

data = data.map(tokenizer_function, batched=True)
data.save_to_disk(config['data_dir'])

## Connect to AzureML workspace and upload data

Now we'll connect to the AzureML workspace and Azure datastore where our dataset will live. We instantiate a connection to the workspace using a configuration file that is automatically provided on the AzureML compute instances but which [we must create](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-environment#workspace) if we run this notebook on our own desktop or laptop machine. 

[Azure datastores](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore.datastore?view=azure-ml-py) can be blob storage, file shares, datalakes, and more. Here we'll use the default, a file share, but any of the other backing services can be used after first [registering them with AzureML](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data#create-and-register-datastores) either through the studio UI or Python SDK.

In [ ]:
workspace = azureml.core.Workspace.from_config()
datastore = workspace.get_default_datastore()

Upload data stored on local disk to the datastore. We are clobbering any prior versions in this upload, assign a UUID subdirectory or other versioning mechanism to retain prior versions of this data within the datastore.

In [ ]:
datastore.upload(src_dir=config['data_dir'], 
                 target_path=config['data_dir'], 
                 overwrite=True
                )

## Register the data as a dataset

Register the dataset with associated metadata describing the task for which it is meant and the model used to tokenize it.

In [ ]:
name = config['task']
description = f"Glue dataset for {config['task']}, tokenized for {config['model']}"
tags = {"task":config['task'], "model": config['model']}
path = datastore.path(config['data_dir'])

amldataset = azureml.core.Dataset.File.from_files(path)
amldataset = amldataset.register(workspace, name, description, tags, True) 

And we'll need to know what version that created in order to use it going forward

In [ ]:
amldataset.version

This shows us the steps to:

- Prepare data before moving it to Azure
- Push data to an Azure datastore accessible by AzureML
- Register the data with AzureML as a dataset